### 주어진 데이터(basic2.csv)에서 "pv"컬럼으로 1일 시차(lag)가 있는 새로운 컬럼을 만들고(예: 1월 2일에는 1월 1일 pv데이터를 넣고, 1월 3일에는 1월 2일 pv데이터를 넣음),새로운 컬럼의 1월 1일은 다음날(1월 2일)데이터로 결측치를 채운 다음, Events가 1이면서 Sales가 1000000이하인 조건에 맞는 새로운 컬럼 합을 구하시오

  
- 데이터셋 : basic2.csv
- 오른쪽 상단 copy&edit 클릭 -> 예상문제 풀이 시작

In [1]:
import pandas as pd

In [11]:
df = pd.read_csv("data/basic2.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB


In [8]:
df = pd.read_csv("data/basic2.csv", parse_dates = ['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB


In [13]:
# 결측치 확인
print(df.isnull().sum())
df.head()

Date       0
Sales      0
PV         0
UV        46
Events     0
dtype: int64


,Date,Sales,PV,UV,Events
0,2022-01-01,22711525,397349,4421.0,1
1,2022-01-02,36779,3969,2289.0,0
2,2022-01-03,13943875,373890,NaN,1
3,2022-01-04,8088593,142054,2157.0,0
4,2022-01-05,2395374,190684,NaN,0


In [28]:
df['PV_new'] = [df['PV'][0]] + df['PV'].to_list()[:-1]

In [30]:
#1일 차이가 나는 시차 특성 만들기
df['previous_PV'] = df['PV'].shift(1) # 하나씩 밀때 이거 사용해 ^^
df.head()

,Date,Sales,PV,UV,Events,previous_PV,PV_new
0,2022-01-01,22711525,397349,4421.0,1,NaN,397349
1,2022-01-02,36779,3969,2289.0,0,397349.0,397349
2,2022-01-03,13943875,373890,NaN,1,3969.0,3969
3,2022-01-04,8088593,142054,2157.0,0,373890.0,373890
4,2022-01-05,2395374,190684,NaN,0,142054.0,142054


In [31]:
# 1일 씩 미뤘음으로 가장 앞이 결측값이 됨 (바로 뒤의 값으로 채움)
df['previous_PV'] = df['previous_PV'].fillna(method = 'bfill')
df.head()

,Date,Sales,PV,UV,Events,previous_PV,PV_new
0,2022-01-01,22711525,397349,4421.0,1,397349.0,397349
1,2022-01-02,36779,3969,2289.0,0,397349.0,397349
2,2022-01-03,13943875,373890,NaN,1,3969.0,3969
3,2022-01-04,8088593,142054,2157.0,0,373890.0,373890
4,2022-01-05,2395374,190684,NaN,0,142054.0,142054


In [32]:
# 조건에 맞는 1일 이전 PV의 합
cond = (df['Events'] == 1) & (df['Sales'] <= 1000000)
print(df[cond]['previous_PV'].sum())

1894876.0
